In [1]:
import pandas as pd
import os

In [2]:
SRC_DIR = r"../downloads/olist"
file_path = r"./artifact/column_description.csv"

In [3]:
df = pd.read_csv(file_path)

# 날짜(시간)이 포함된 데이터

In [4]:
time_columns = []
for column_name in df.column_name:
    if 'date' in column_name or 'time' in column_name or '_at' in column_name:
        time_columns.append(column_name)
time_metadata = df[df.column_name.isin(time_columns)]
time_metadata

,file_name,column_name,description
14,olist_order_items_dataset.csv,shipping_limit_date,판매자가 상품을 배송사에 전달해야 하는 기한
27,olist_order_reviews_dataset.csv,review_creation_date,리뷰가 작성된 날짜
28,olist_order_reviews_dataset.csv,review_answer_timestamp,리뷰에 대한 답변이 작성된 날짜 및 시간
32,olist_orders_dataset.csv,order_purchase_timestamp,주문이 생성된 날짜 및 시간
33,olist_orders_dataset.csv,order_approved_at,주문이 승인된 날짜 및 시간
34,olist_orders_dataset.csv,order_delivered_carrier_date,주문이 배송사에 전달된 날짜
35,olist_orders_dataset.csv,order_delivered_customer_date,주문이 고객에게 배송 완료된 날짜
36,olist_orders_dataset.csv,order_estimated_delivery_date,주문의 예상 배송 날짜


# 시간과 관련된 데이터로 부터 확인할 수 있는 가정
Seller
- [COULD] 주문 생성 시간을 확인할 수 있다. `order_purchase_timestamp`
- [MUST] 주문 승인한다. `order_approved_at`
- [COULD] 예상 배송 날짜를 확인할 수 있다. `order_estimated_delivery_date`
- [MUST] 판매자는 배송사에 기한 내 보내야한다. `shipping_limit_date`
- [COULD] 배송사에 전달된 날짜을 확인할 수 있다. `order_delivered_carrier_date`
- [COULD] 배송완료 날짜를 확인할 수 있다 `order_delivered_customer_date`
- [MUST] 고객의 리뷰에 답글을 남겨야만한다. `review_answer_timestamp`

Olist
- [MUST] 주문 생성 시간을 생성해야한다. `order_purchase_timestamp`
- [MUST] 주문 승인 시간을 생성해야한다. `order_approved_at`
- [MUST] 예상 배송 날짜를 생성해야한다. `order_estimated_delivery_date`
- [MUST] 배송사에 전달된 날짜를 생성해야한다. `order_delivered_carrier_date`
- [MUST] 배송완료 날짜를 생성해야한다. `order_delivered_customer_date`

Customer
- [COULD] 주문 생성 시간을 확인할 수 있다. `order_purchase_timestamp`
- [COULD] 주문 승인 시간을 확인할 수 있다. `order_approved_at`
- [COULD] 배송사에 전달된 날짜를 확인할 수 있다. `order_delivered_carrier_date`
- [COULD] 배송완료 날짜를 확인할 수 있다. `order_delivered_customer_date`
- [COULD] 예상 배송 날짜를 확인할 수 있다. `order_estimated_delivery_date`
- [COULD] 리뷰를 남길 수 있다. `review_creation_date`

# Dataset에서 날짜 범위 확인

In [5]:
time_min = pd.Timestamp.max
time_max = pd.Timestamp.min
time_src = time_metadata.groupby('file_name')['column_name'].apply(list)
for file_name, column_list in time_src.items():
    file_path = os.path.join(SRC_DIR, file_name)
    tmp_df = pd.read_csv(file_path)
    for column in column_list:
        tmp_time_min = pd.to_datetime(tmp_df[column]).min()
        tmp_time_max = pd.to_datetime(tmp_df[column]).max()
        
        time_min = min(time_min, tmp_time_min)
        time_max = max(time_max, tmp_time_max)

print("데이터에서 가장 과거의 날짜: ", time_min)
print("데이터에서 가장 최근의 날짜: ", time_max)

데이터에서 가장 과거의 날짜:  2016-09-04 21:15:19
데이터에서 가장 최근의 날짜:  2020-04-09 22:35:08
